In [2]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

from tqdm import tqdm
import pickle

In [141]:
url = 'https://bigbangtrans.wordpress.com/'
response = requests.get(url)
if response.status_code != 200:
    raise Exception(f'wrong status code {response.status_code}')
else:
    bs = BeautifulSoup(response.text,"html")
    temps_href = bs.findAll("li", {"class": "page_item"})
    temps_href = temps_href[1:]
    temps_href = [[tmp.a['href'], tmp.text] for tmp in temps_href]
    
out_list = []

for url_h, name in tqdm(temps_href):
    response_h = requests.get(url_h)
    bs_h = BeautifulSoup(response_h.text,"html")
    temps = bs_h.findAll("p", {"class": "MsoNormal"})
    temps = [temp.text for temp in temps]
    
    if len(temps) == 0:
        parent = bs_h.find("div", {"class": "entrytext"})
        temps = parent.findAll('p')
        temps = [temp.text for temp in temps]
    
    for i, temp in enumerate(temps):
        if temp.find('Sheldon:') == 0:
            if i == 0: continue
            if i == 1:
                out_list.append([name, '', temps[i-1], temp])
            else:
                out_list.append([name, temps[i-2], temps[i-1], temp])

out_df = pd.DataFrame(out_list, columns=['name', 'context', 'question', 'answer'])
out_df['label'] = 1
df2 = out_df.copy(deep=True)
df2['answer'] = df2['answer'].iloc[::-1].reset_index(drop=True)
df2['label'] = 0
df_all = pd.concat([out_df, df2])
df_all.to_csv('data/BBT_sheldon_all.csv', index=False)

dataset_dict = df_all.to_dict(orient='records')
with open('data/dataset.pkl', 'wb') as file:
    pickle.dump(dataset_dict, file)

100%|██████████| 231/231 [03:39<00:00,  1.05it/s]
